### Downsample image planes for large images (ie. COLM)


In [45]:
import os

import tkinter as tk
import tkinter.filedialog as fdialog
from tkinter import simpledialog
# this is the gui for finding directory and files

import skimage
from skimage import io
from skimage import transform as tf

#import matplotlib.pyplot as plt

from tqdm import tqdm


In [7]:
# Select the directory containing already stitched planes from stitchit (Rob code)
in_dir=fdialog.askdirectory(initialdir='Z:\\', title='select the folder containing single section images to be further downsampled')                               

# User input required for raw xyz resolution and goal dimension
xy = simpledialog.askfloat("Input", "What is the x and y resolution in um?", minvalue=0.0, maxvalue=100)
#z = simpledialog.askfloat("Input", "What is the z resolution in um?",minvalue=0.0, maxvalue=100)
goal_xy = simpledialog.askfloat("Input", "What do you want to downsample the xy pixel size to '(in um)' ?",minvalue=10, maxvalue=100)

outdir = fdialog.askdirectory(title='Please select the output directory')

In [8]:
# Code obtained from Lucas for reference
# for own downsampling method, use skimage.transform.resize
def get_dir(path):
    tiffs = [os.path.join(path, f) for f in os.listdir(path) if f[0] != '.']
    return sorted(tiffs)

def downscale_imgs(in_folder, out_folder, factor=(10,10,1)):
    images = get_dir(in_folder)
    
    for _,item in enumerate(tqdm(images)):
        img = cv2.imread(item, -1) #black/white
        out_img = block_reduce(img, block_size=factor, func=np.mean)
        out_img = Image.fromarray(out_img)
        out_img.save(out_folder + "//" + "ds_" + os.path.basename(item))

In [11]:
images = get_dir(in_dir)

In [32]:
im = io.imread(images[0])
rows,cols= im.shape
print (im.shape) 


(13824, 16214)


In [33]:
ratio=goal_xy/xy

new_rows=int(rows/ratio)
new_cols=int(cols/ratio)

print(f'Original image resolution is {xy,xy} um, new image pixel size is {goal_xy} um')
print(f'Original image size is {rows,cols} pixels, new image size is {new_rows, new_cols} pixels')
print(f'Downsample factor is {ratio:.2f}')

Original image resolution is (0.6, 0.6) um, new image pixel size is 25.0 um
Original image size is (13824, 16214) pixels, new image size is (331, 389) pixels
Downsample factor is 41.67


In [36]:
new_xyplane=tf.resize(im[:,:],[new_rows, new_cols], order=1, anti_aliasing=1)

In [46]:
for _,item in enumerate(tqdm(images[0:5])):
    im= io.imread(item)
    new_xyplane=tf.resize(im[:,:],[new_rows, new_cols], order=1, anti_aliasing=1)
    imguint16=skimage.img_as_uint(new_xyplane)
    io.imsave(outdir + "//" + "ds_" + os.path.basename(item), imguint16)


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]C:\Users\liu\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\util\dtype.py:135: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\liu\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\io\_io.py:141: UserWarning: D:/AL233_25umxy//ds_000000_000000_000000.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
 20%|████████████████▊                                                                   | 1/5 [00:55<03:43, 55.78s/it]C:\Users\liu\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\io\_io.py:141: UserWarning: D:/AL233_25umxy//ds_000000_000000_000030.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
 40%|█████████████████████████████████▌                                                  | 2/5 [01:51<02:47, 55.68s/it]C:\User

In [41]:
outname=outdir + "//" + "ds_" + os.path.basename(images[0])
outname

'D:/AL233_25umxy//ds_000000_000000_000000.tif'

41.0

25.0

0.6